In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import os

import seaborn as sns

from scipy import sparse

import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, leaves_list

plt.rcParams.update({
    "axes.titlesize": 26,
    "axes.labelsize": 18,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
})

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
from pygam import LinearGAM, s
import pygam

In [ ]:
mouse_base_path = '/mouse/base/path/'

mouse_h5ad_file = 'mouse.h5ad'
mouse_h5ad_path = os.path.join(mouse_base_path,mouse_h5ad_file)

human_base_path = '/human/base/path/
human_h5ad_file = 'human.h5ad'
human_h5ad_path = os.path.join(human_base_path,human_h5ad_file)

mi_base_path = '.'
mi_genes_file = './pseudotime_orthologs.csv'
mi_genes_path = os.path.join(mi_base_path, mi_genes_file)

In [ ]:
adata_mouse = sc.read_h5ad(mouse_h5ad_path)

In [ ]:
adata_human = sc.read_h5ad(human_h5ad_path)

In [ ]:
mi_genes = pd.read_csv(mi_genes_path,  index_col = 0)

In [ ]:
adata_mouse_mi = adata_mouse[:,mouse_genes].copy()

In [ ]:
adata = adata_mouse_mi.copy()

# pull pseudotime + expression
pt = adata.obs["dpt_pseudotime"].astype(float).values
mask = np.isfinite(pt)
expr_df = sc.get.obs_df(adata[mask], keys=list(adata.var_names), layer="log1p")
expr_df["pseudotime"] = pt[mask]

pt_grid = np.linspace(expr_df["pseudotime"].min(), expr_df["pseudotime"].max(), 800)

# fit GAM for each gene
smoothed = {}
x = expr_df["pseudotime"].values
for gene in adata.var_names:
    y = expr_df[gene].values
    gam = LinearGAM(s(0, n_splines=20)).fit(x, y)
    smoothed[gene] = gam.predict(pt_grid)

heatmap_df = pd.DataFrame(smoothed, index=pt_grid).T  # genes by pseudotime grid

# quantile-normalize
low = heatmap_df.quantile(0.01, axis=1)
high = heatmap_df.quantile(0.99, axis=1)
robust = (
    heatmap_df
    .sub(low, axis=0)
    .div((high - low).replace(0, np.nan), axis=0)
    .clip(0, 1)
    .fillna(0)
)

# sort genes by peak pseudotime
peak_idx = robust.values.argmax(axis=1)
order = np.argsort(peak_idx)
robust = robust.iloc[order]

# plot arrays
norm_epxr = robust.values

# heatmap
plt.figure(figsize=(10, 16))
im = plt.imshow(
    norm_epxr,
    aspect="auto",
    interpolation="bilinear",
    cmap="magma",d
    origin="upper"
)

plt.colorbar(im, label="normalized expression (0–1)")
plt.xlabel("pseudotime")
plt.ylabel("genes")
plt.title("GAM-smoothed expression (quantile-normalized)")

plt.yticks(range(len(robust)), robust.index, fontsize=6)
plt.xticks([0, int(norm_epxr.shape[1]/2), norm_epxr.shape[1]-1], ["0.0", "0.5", "1.0"])

plt.tight_layout()
plt.savefig("./mouse_pseudotime_heatmap.png", dpi=300,
            bbox_inches="tight")
plt.show()

In [ ]:
adata_human_mi = adata_human[:,human_genes].copy()

In [ ]:
adata = adata_human_mi.copy()

# pull pseudotime + expression
pt = adata.obs["dpt_pseudotime"].astype(float).values
mask = np.isfinite(pt)
expr_df = sc.get.obs_df(adata[mask], keys=list(adata.var_names), layer="log1p")
expr_df["pseudotime"] = pt[mask]

pt_grid = np.linspace(expr_df["pseudotime"].min(), expr_df["pseudotime"].max(), 800)

# fit GAM for each gene
smoothed = {}
x = expr_df["pseudotime"].values
for gene in adata.var_names:
    y = expr_df[gene].values
    gam = LinearGAM(s(0, n_splines=20)).fit(x, y)
    smoothed[gene] = gam.predict(pt_grid)

heatmap_df = pd.DataFrame(smoothed, index=pt_grid).T  # genes by pseudotime grid

# quantile-normalize
low = heatmap_df.quantile(0.01, axis=1)
high = heatmap_df.quantile(0.99, axis=1)
robust = (
    heatmap_df
    .sub(low, axis=0)
    .div((high - low).replace(0, np.nan), axis=0)
    .clip(0, 1)
    .fillna(0)
)

# sort genes by peak pseudotime
peak_idx = robust.values.argmax(axis=1)
order = np.argsort(peak_idx)
robust = robust.iloc[order]

# plot arrays
norm_epxr = robust.values

# heatmap
plt.figure(figsize=(10, 16))
im = plt.imshow(
    norm_epxr,
    aspect="auto",
    interpolation="bilinear",
    cmap="magma",d
    origin="upper"
)

plt.colorbar(im, label="normalized expression (0–1)")
plt.xlabel("pseudotime")
plt.ylabel("genes")
plt.title("GAM-smoothed expression (quantile-normalized)")

plt.yticks(range(len(robust)), robust.index, fontsize=6)
plt.xticks([0, int(norm_epxr.shape[1]/2), norm_epxr.shape[1]-1], ["0.0", "0.5", "1.0"])

plt.tight_layout()
plt.savefig("./human_pseudotime_heatmap.png", dpi=300,
            bbox_inches="tight")
plt.show()